In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from keras.utils import to_categorical
import tensorflow as tf
import time
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from keras import regularizers
from sklearn.model_selection import train_test_split
import cv2 as cv
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset = pd.read_csv('/content/drive/My Drive/Colab Notebooks/movie_poster.csv')
# /content/drive/My Drive/Colab Notebooks/movie_poster.csv

In [ ]:
dataset = dataset*1
#converting bool to int
dataset.drop(dataset.columns[[1,2,3,4]], axis=1, inplace=True)
print(dataset)

            tid  action  scifi  drama  romance
0     tt0120737       1      0      0        0
1     tt0167260       1      0      0        0
2     tt0167261       1      0      0        0
3     tt0169102       0      0      1        0
4     tt0211915       0      0      0        1
...         ...     ...    ...    ...      ...
2450  tt1692486       0      0      1        0
2451  tt1695360       1      0      0        0
2452  tt1696202       1      0      0        0
2453  tt1699202       0      0      1        0
2454  tt1699513       0      0      1        0

[2455 rows x 5 columns]


In [ ]:
dataset_clean = dataset
pic = cv.imread('/content/drive/My Drive/Colab Notebooks/posters/tt1692486.jpg')

In [ ]:
#print(pic)

In [ ]:
df = dataset[['tid']]
df = np.array(df)
s = ''
for j in range (df.shape[0]):
    df = dataset[['tid']]
    df = np.array(df)
    v =  '/content/drive/My Drive/Colab Notebooks/posters/'+ df[j][0] + '.jpg'
    img = cv.imread(v)
    imgsize = 320
    try:
        img = cv.resize(img, (imgsize, imgsize))
    except:
        s = df[j][0]
        dataset_clean = dataset_clean[~dataset_clean['tid'].isin([s])]

In [ ]:
print(dataset_clean)

            tid  action  scifi  drama  romance
0     tt0120737       1      0      0        0
1     tt0167260       1      0      0        0
2     tt0167261       1      0      0        0
3     tt0169102       0      0      1        0
4     tt0211915       0      0      0        1
...         ...     ...    ...    ...      ...
2450  tt1692486       0      0      1        0
2451  tt1695360       1      0      0        0
2452  tt1696202       1      0      0        0
2453  tt1699202       0      0      1        0
2454  tt1699513       0      0      1        0

[2436 rows x 5 columns]


In [ ]:
train, test = train_test_split(dataset_clean, test_size=0.2)

In [ ]:
df1 = train[['action','scifi','drama','romance']]
df = train[['tid']]
df = np.array(df)
df_test = test[['tid']]
df_test = np.array(df_test)
df_test1 = test[['action','scifi','drama','romance']]

In [ ]:
y_train = np.array(df1)
y_test = np.array(df_test1)

In [ ]:
def ExtractImage(x):
    v = '/content/drive/My Drive/Colab Notebooks/posters/'+ x + '.jpg'
    v_list = []
    for i in range(v.shape[0]):
        s = v[i][0]
        #print(s)
        img = cv.imread(s)
        imgsize = 360
        img = cv.resize(img, (imgsize, imgsize))
        v_list.append(img)
    images = np.array(v_list)
    return images

In [ ]:
x_train = ExtractImage(df)

In [ ]:
x_test = ExtractImage(df_test)

In [ ]:
x_train.shape

(1948, 360, 360, 3)

In [ ]:
x_test.shape

(488, 360, 360, 3)

In [ ]:
model = Sequential()
model.add(Conv2D(23, kernel_size=(3,3),padding= 'same',data_format='channels_last', input_shape=(360,360,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(45, kernel_size=(3,3),padding= 'same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(60, kernel_size=(1,1),padding= 'same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128,activation= tf.nn.relu, kernel_regularizer=regularizers.l2(0.01),activity_regularizer=regularizers.l1(0.01)))
model.add(Dropout(0.5))
model.add(Dense(64,activation= tf.nn.relu, kernel_regularizer=regularizers.l2(0.01),activity_regularizer=regularizers.l1(0.01)))
model.add(Dropout(0.2))
model.add(Dense(42,activation= tf.nn.relu))
model.add(Dense(4,activation= tf.nn.sigmoid))






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [ ]:
#datagen.fit(x_train)

In [ ]:
train_generator= datagen.flow(x_train, y_train, batch_size=32)


In [ ]:
batch_size = 32


In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=50)




Epoch 1/50


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:724: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '







62/62 [==============================] - 58s 938ms/step - loss: 149.7288 - acc: 0.7560
Epoch 2/50
62/62 [==============================] - 49s 784ms/step - loss: 9.2325 - acc: 0.7864
Epoch 3/50
62/62 [==============================] - 49s 790ms/step - loss: 8.9943 - acc: 0.7880
Epoch 4/50
62/62 [==============================] - 49s 786ms/step - loss: 8.7658 - acc: 0.7856
Epoch 5/50
62/62 [==============================] - 49s 791ms/step - loss: 8.5470 - acc: 0.7850
Epoch 6/50
62/62 [==============================] - 49s 794ms/step - loss: 8.3390 - acc: 0.7856
Epoch 7/50
62/62 [==============================] - 49s 795ms/step - loss: 8.1406 - acc: 0.7899
Epoch 8/50
62/62 [==============================] - 49s 792ms/step - loss: 7.9613 - acc: 0.7842
Epoch 9/50
62/62 [==============================] - 49s 793ms/step - loss: 7.7856 - acc: 0.7844
Epoch 10/50
62/62 [==============================] - 49s 784ms/step - loss: 7.6089 - acc: 0.7898
Epoch 11/50
62/62 [========================

In [ ]:
model.evaluate(x_test,y_test)

488/488 [==============================] - 1s 2ms/step


[3.0439083224437278, 0.7817622950819673]